In [ ]:
import os
from torchvision.transforms import ToTensor
from skimage import io
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms, models
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split, TensorDataset, Dataset
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
from torch import optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/oxford-102-flowers.tgz"
download_url(dataset_url, '.')

with tarfile.open('./oxford-102-flowers.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

img_path='./data/oxford-102-flowers/jpg/image_00001.jpg'

In [ ]:
class flowersmodel(Dataset):
  def __init__(self,excel_file,root_dir,transform=None):
    self.annotations=pd. read_csv(excel_file,delimiter=' ')
    self.root_dir=root_dir
    self.transform=transform
  def __len__(self):
    return len(self.annotations)
  def __getitem__(self,index):
    img_path=os.path.join(self.root_dir,self.annotations.iloc[index,0])
    image=io.imread(img_path)
    y_label=torch.tensor(self.annotations.iloc[index,1])
    image=Image.open(img_path).resize((300,300),resample=0)
    if self.transform:
      image=self.transform(image)
    return (image,y_label)

In [ ]:
from torchvision.transforms import ToTensor


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(75),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'testing': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


train_dataset=flowersmodel('./data/oxford-102-flowers/train.txt',root_dir='./data/oxford-102-flowers',transform=data_transforms['train'])
test_dataset=flowersmodel('./data/oxford-102-flowers/test.txt',root_dir='./data/oxford-102-flowers',transform=data_transforms['testing'])
val_dataset=flowersmodel('./data/oxford-102-flowers/valid.txt',root_dir='./data/oxford-102-flowers',transform=data_transforms['validation'])





In [ ]:
from tqdm.notebook import tqdm
import time

def compute_accuracy(model, loader):
    total_correct = 0

    model.cuda()
    model.eval()
    for inputs, labels in tqdm(loader, leave=False):
        inputs, labels = inputs.cuda(), labels.cuda()
        output = model(inputs)
        _, pred = torch.max(output, 1)
        for d in zip(pred, labels):
          if d[0].item() == d[1].item():
            total_correct += 1
    return total_correct / len(loader.dataset)

def train(model, train_loader, val_loader, num_epochs, criterion, optimizer, path = None, scheduler = None):
    print('beginning to train model')
    if path and not os.path.exists(path):
      os.makedirs(path)
    model.cuda()
    for epoch in tqdm(range(1, num_epochs + 1)):
        model.train()
        total_loss = 0
        start_time = time.perf_counter()
        for inputs, labels in tqdm(train_loader, leave=False):
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
        if path:
          torch.save(model.state_dict(), f'{path}/model_ep_{epoch:02d}.pth')
        end_time = time.perf_counter()
        duration = end_time - start_time

        train_acc = compute_accuracy(model, val_loader)



        current_lr = optimizer.param_groups[0]['lr']

        if scheduler and current_lr > 5e-5:
            scheduler.step()

        print(f'epoch {epoch:2}',
              f'loss: {total_loss:.3f}',
              f'time: {duration:.3f}',
              f'val acc: {train_acc:.4f}',
              sep='\t')



In [ ]:
dataloaders = {
    'training' : torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True),
    'testing' : torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False),
    'validation' : torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True)
}

In [ ]:
class EfficientNetB1(nn.Module):
    def __init__(self, n_classes=102, learnable_modules = ('classifier.1',)):
        super().__init__()
        self.efficientnet_b1 = models.efficientnet_b1(weights = 'DEFAULT')
        self.efficientnet_b1.classifier[1] = nn.Linear(self.efficientnet_b1.classifier[1].in_features, n_classes)
        self.efficientnet_b1.requires_grad_(False)
        modules = dict(self.efficientnet_b1.named_modules())
        for name in learnable_modules:
            modules[name].requires_grad_(True)

    def forward(self, x):
        return F.log_softmax(self.efficientnet_b1(x), dim = 1)

In [ ]:
model = nn.DataParallel(EfficientNetB1(n_classes = 102, learnable_modules = ('features.5.2',
                                                                             'features.6',
                                                                             'features.7',
                                                                             'features.8',
                                                                             'classifier')))
epochs = 25
PATH = 'morgana_christmas_flowers102'
cat_to_name = {"21": "fire lily", "3": "canterbury bells", "45": "bolero deep blue", "1": "pink primrose", "34": "mexican aster", "27": "prince of wales feathers", "7": "moon orchid", "16": "globe-flower", "25": "grape hyacinth", "26": "corn poppy", "79": "toad lily", "39": "siam tulip", "24": "red ginger", "67": "spring crocus", "35": "alpine sea holly", "32": "garden phlox", "10": "globe thistle", "6": "tiger lily", "93": "ball moss", "33": "love in the mist", "9": "monkshood", "102": "blackberry lily", "14": "spear thistle", "19": "balloon flower", "100": "blanket flower", "13": "king protea", "49": "oxeye daisy", "15": "yellow iris", "61": "cautleya spicata", "31": "carnation", "64": "silverbush", "68": "bearded iris", "63": "black-eyed susan", "69": "windflower", "62": "japanese anemone", "20": "giant white arum lily", "38": "great masterwort", "4": "sweet pea", "86": "tree mallow", "101": "trumpet creeper", "42": "daffodil", "22": "pincushion flower", "2": "hard-leaved pocket orchid", "54": "sunflower", "66": "osteospermum", "70": "tree poppy", "85": "desert-rose", "99": "bromelia", "87": "magnolia", "5": "english marigold", "92": "bee balm", "28": "stemless gentian", "97": "mallow", "57": "gaura", "40": "lenten rose", "47": "marigold", "59": "orange dahlia", "48": "buttercup", "55": "pelargonium", "36": "ruby-lipped cattleya", "91": "hippeastrum", "29": "artichoke", "71": "gazania", "90": "canna lily", "18": "peruvian lily", "98": "mexican petunia", "8": "bird of paradise", "30": "sweet william", "17": "purple coneflower", "52": "wild pansy", "84": "columbine", "12": "colt's foot", "11": "snapdragon", "96": "camellia", "23": "fritillary", "50": "common dandelion", "44": "poinsettia", "53": "primula", "72": "azalea", "65": "californian poppy", "80": "anthurium", "76": "morning glory", "37": "cape flower", "56": "bishop of llandaff", "60": "pink-yellow dahlia", "82": "clematis", "58": "geranium", "75": "thorn apple", "41": "barbeton daisy", "95": "bougainvillea", "43": "sword lily", "83": "hibiscus", "78": "lotus lotus", "88": "cyclamen", "94": "foxglove", "81": "frangipani", "74": "rose", "89": "watercress", "73": "water lily", "46": "wallflower", "77": "passion flower", "51": "petunia"}
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = [{'params': model.module.efficientnet_b1.features[5][2].parameters()},
                                       {'params': model.module.efficientnet_b1.features[6].parameters()},
                                       {'params': model.module.efficientnet_b1.features[7].parameters()},
                                       {'params': model.module.efficientnet_b1.features[8].parameters()},
                                       {'params': model.module.efficientnet_b1.classifier.parameters(), 'lr': 1e-3}],
                             lr = 1e-4,
                             weight_decay = 1e-4)
train(model, dataloaders['training'], dataloaders['validation'], epochs, criterion, optimizer, PATH)

In [ ]:

from torchsummary import summary
model = nn.DataParallel(EfficientNetB1(n_classes = 102, learnable_modules = ('features.5.2',
                                                                             'features.6',
                                                                             'features.7',
                                                                             'features.8',
                                                                             'classifier')))
model.cuda()
model.load_state_dict(torch.load('/content/drive/MyDrive/MORGANA_EFFB1_NET.pth'))



In [ ]:
category = {}
cat_to_name = {"21": "fire lily", "3": "canterbury bells", "45": "bolero deep blue", "1": "pink primrose", "34": "mexican aster", "27": "prince of wales feathers", "7": "moon orchid", "16": "globe-flower", "25": "grape hyacinth", "26": "corn poppy", "79": "toad lily", "39": "siam tulip", "24": "red ginger", "67": "spring crocus", "35": "alpine sea holly", "32": "garden phlox", "10": "globe thistle", "6": "tiger lily", "93": "ball moss", "33": "love in the mist", "9": "monkshood", "102": "blackberry lily", "14": "spear thistle", "19": "balloon flower", "100": "blanket flower", "13": "king protea", "49": "oxeye daisy", "15": "yellow iris", "61": "cautleya spicata", "31": "carnation", "64": "silverbush", "68": "bearded iris", "63": "black-eyed susan", "69": "windflower", "62": "japanese anemone", "20": "giant white arum lily", "38": "great masterwort", "4": "sweet pea", "86": "tree mallow", "101": "trumpet creeper", "42": "daffodil", "22": "pincushion flower", "2": "hard-leaved pocket orchid", "54": "sunflower", "66": "osteospermum", "70": "tree poppy", "85": "desert-rose", "99": "bromelia", "87": "magnolia", "5": "english marigold", "92": "bee balm", "28": "stemless gentian", "97": "mallow", "57": "gaura", "40": "lenten rose", "47": "marigold", "59": "orange dahlia", "48": "buttercup", "55": "pelargonium", "36": "ruby-lipped cattleya", "91": "hippeastrum", "29": "artichoke", "71": "gazania", "90": "canna lily", "18": "peruvian lily", "98": "mexican petunia", "8": "bird of paradise", "30": "sweet william", "17": "purple coneflower", "52": "wild pansy", "84": "columbine", "12": "colt's foot", "11": "snapdragon", "96": "camellia", "23": "fritillary", "50": "common dandelion", "44": "poinsettia", "53": "primula", "72": "azalea", "65": "californian poppy", "80": "anthurium", "76": "morning glory", "37": "cape flower", "56": "bishop of llandaff", "60": "pink-yellow dahlia", "82": "clematis", "58": "geranium", "75": "thorn apple", "41": "barbeton daisy", "95": "bougainvillea", "43": "sword lily", "83": "hibiscus", "78": "lotus lotus", "88": "cyclamen", "94": "foxglove", "81": "frangipani", "74": "rose", "89": "watercress", "73": "water lily", "46": "wallflower", "77": "passion flower", "51": "petunia"}
for key in cat_to_name:
  k = int(key) - 1
  category[f"{k}"] = cat_to_name[f'{key}']

print(category)



In [ ]:
PATH = "" #path to a dowloaded .jpg test image
image = Image.open(PATH).resize((300,300),resample=0)
plt.imshow(image)
plt.show()
img_trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
image = img_trans(image)
image = torch.reshape(image, (1, 3, 224, 224))
plt.imshow(torch.reshape(image, (224, 224, 3)))
plt.show()
model.eval()
model.cuda()
image = image.cuda()
output = model(image)
_, pred = torch.max(output, 1)
print(pred)
print(category["75"])
print("MODEL PREDICTION: ", category[f"{pred.item()}"])


In [ ]:


correct = 0
count = 0
for inputs, labels in dataloaders['testing']:
  model.cuda()
  output = model(inputs)
  _, pred = torch.max(output, 1)
  predicted = pred.item()
  ground_truth = labels.item()
  print(ground_truth, predicted)